In [1]:
#pip install websockets
#pip install nest_asyncio
#%autoawait asyncio
#!pip install simplet5

In [2]:
import pyaudio
import wave
import asyncio
from pydub import AudioSegment
import json
import json
import websockets
import smtplib
import ssl
from email.message import EmailMessage
from vosk import Model, KaldiRecognizer
from datetime import datetime
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import glob
from simplet5 import SimpleT5
import time
import os
from datetime import datetime

FRAME_RATE = 16000
model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)
# Set the audio parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100  # Sample rate (samples per second)
RECORD_SECONDS = 10  # Duration of the recording in seconds
OUTPUT_FILENAME_TEMPLATE = "meeting_{}.wav"
loop_counter =1
summarized_content=''
startTime=''
endTime=''

tokenizer=AutoTokenizer.from_pretrained('t5-base')
model=AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

# Load the model
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.load_model("t5","outputs/simplet5-epoch-1-train-loss-1.1844-val-loss-1.2571", use_gpu=False)

Global seed set to 42
C:\Users\User\anaconda3\lib\site-packages\transformers\models\auto\modeling_auto.py:803: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [3]:
async def startRecording(stop_flag):
    audio = pyaudio.PyAudio()

    loop_counter = 1
    while not stop_flag.is_set():
        print(stop_flag.is_set())

        # Create an audio stream
        stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=1024)

        print(f"Recording loop {loop_counter} - {RECORD_SECONDS} seconds...")

        frames = []

        # Record audio
        for _ in range(0, int(RATE / 1024 * RECORD_SECONDS)):
            if stop_flag.is_set():
                break
            data = stream.read(1024)
            frames.append(data)

            # Check if the stop flag is set during recording
        if stop_flag.is_set():
                break   

        print(f"Finished recording loop {loop_counter}.")

        # Stop and close the audio stream
        stream.stop_stream()
        stream.close()

        # Save the recorded audio to a WAV file
        output_filename = OUTPUT_FILENAME_TEMPLATE.format(loop_counter)
        with wave.open(output_filename, 'wb') as wf:
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(audio.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(frames))

        print(f"Audio saved to {output_filename}")

        loop_counter += 1
        await asyncio.sleep(1)

    # Stop and close the PyAudio instance when the recording is stopped
    audio.terminate()
    print("Recording stopped")

In [4]:
# async def pingWebSocket(stop_flag1, websocket):
#     while not stop_flag1.is_set():
#         print("Sending loveeee")
#         time.sleep(5)
#         response = {"message": "pinging"}
#         await websocket.send(json.dumps(response))
#         await asyncio.sleep(1)


In [5]:
def count_wav_files():
    wav_files = glob.glob(f"*.wav")
    return len(wav_files)

def convertAudioToText():
    out = ''
    loop_counter = count_wav_files()
    print("Converting started")
    for i in range(1, loop_counter+1):  # Assuming n is the total number of recordings
        # Construct the filename
        filename = f"meeting_{i}.wav"
        print("Converting file : ",  filename)
        # Load the audio file
        wav = AudioSegment.from_wav(filename)

        # Set the number of channels and frame rate
        wav = wav.set_channels(CHANNELS)
        wav = wav.set_frame_rate(FRAME_RATE)

        rec.AcceptWaveform(wav.raw_data)
        result = rec.Result()
        text = json.loads(result)["text"]
        out = out + '\n' + text
    return out

In [6]:
def summarizer(text_to_summarize):
    generated_text = model.predict(text_to_summarize)
    print("Generated Text:", generated_text)
    return generated_text

In [7]:
def calculate_duration(start_time, end_time):
    print("StartTime", start_time, "EndTime", end_time)
    # Convert string representations of time to datetime objects
    start_datetime = datetime.strptime(start_time, "%H:%M:%S")
    end_datetime = datetime.strptime(end_time, "%H:%M:%S")

    # Calculate the duration
    duration = end_datetime - start_datetime

    return str(duration)

In [8]:
def getCurrentTime():

    # Get the current date and time
    current_datetime = datetime.now()

    # Format the date and time as a string
    formatted_datetime = current_datetime.strftime("%H:%M:%S")

    print(f"The current date and time is: {formatted_datetime}")
    return formatted_datetime

In [9]:
def send_email(to_email, subject, body):
    # Set your email and password
    sender_email = 'kothapallipriyanka05@gmail.com'
    sender_password = 'ibfk lefu zzfg gcjw'

    # Set up the MIME
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = sender_email
    message["Cc"] = ','.join(to_email)
    message['Subject'] = subject

# Split the string into a list using commas as the separator
    attendees = str(len(to_email))

    duration = calculate_duration(startTime, endTime)
    current_date = datetime.now().date()

# Convert the date to a string with a specific format
    date_string = datetime.now().strftime("%Y-%m-%d")
    print("Sender mail", to_email)
    print("start", startTime)
    print("end", endTime)
    print("date", date_string)
    print("duration", duration)

    html_body = MIMEText(f"""
<div style="text-align:center;margin-top="100px"><table style ="border-collapse: collapse; width: 60%;margin-left:20%"">
    <tr>
        <th style="border: 1px solid #dddddd; text-align: center; padding: 8px;">Recording Done by</th>
        <td style="border: 1px solid #dddddd; text-align: center; padding: 8px;">{sender_email}</td>
    </tr>
    <tr>
        <th style="border: 1px solid #dddddd; text-align: center; padding: 8px;">Start Time</th>
        <td style="border: 1px solid #dddddd; text-align: center; padding: 8px;">{startTime}</td>
    </tr>
    <tr>
        <th style="border: 1px solid #dddddd; text-align: center; padding: 8px;">End Time</th>
        <td style="border: 1px solid #dddddd; text-align: center; padding: 8px;">{endTime}</td>
    </tr>
    <tr>
        <th style="border: 1px solid #dddddd; text-align: center; padding: 8px;">Date</th>
        <td style="border: 1px solid #dddddd; text-align: center; padding: 8px;">{date_string}</td>
    </tr>
    <tr>
        <th style="border: 1px solid #dddddd; text-align: center; padding: 8px;">Duration</th>
        <td style="border: 1px solid #dddddd; text-align: center; padding: 8px;">{duration}</td>
    </tr>
    <tr>
        <th style="border: 1px solid #dddddd; text-align: center; padding: 8px;">Attendees</th>
        <td style="border: 1px solid #dddddd; text-align: center; padding: 8px;">{attendees}</td>
    </tr>
</table>
<p style="font-family: 'Arial', sans-serif; font-size: 16px; line-height: 1.6; color: #333; margin-bottom: 20px;">
    <b>MOM of the meeting:</b>
</p>
<p style="font-family: 'Arial', sans-serif; font-size: 16px; line-height: 1.6; color: #333; margin-bottom: 20px;">{body}</p>
<p style="font-family: 'Arial', sans-serif; font-size: 10px; line-height: 1.6; color: #333;margin-top:10%"><i>
    This summarization is done by Meeting Minder</i>
</p></body>""", "html")

    message.attach(html_body)

    try:
        # Connect to the SMTP server (for Gmail)
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            # Login to your email account
            server.login(sender_email, sender_password)
            # Send the email
            server.sendmail(sender_email, to_email, message.as_string())
            print("Email sent successfully.")
    except Exception as e:
        print(f"Error: {e}")

# List of emails to send

def sendMail(mails, summarized_text):
    # Send the email
    send_email(mails, 'MOM of the Meeting from ' + startTime+ ' To '+ endTime, summarized_text[0])

In [10]:
def delete_audio_files():
    directory = os.getcwd()
    file_list = os.listdir(directory)

    for file_name in file_list:
        if file_name.endswith(".wav"):
            file_path = os.path.join(directory, file_name)
            os.remove(file_path)

In [11]:
async def on_message(websocket, path, stop_flag):
    async for message in websocket:
        print(f"Received message from extension: {message}")
        
        # Process the message from the extension as needed
        # Parse the JSON string into a Python dictionary
        json_data = json.loads(message)

        # Access the value associated with the "action" key
        action_value = json_data['action']
        print(action_value)
        # Send feedback to the extension
        if action_value =="startRecording":
            global startTime
            startTime = getCurrentTime()
            asyncio.create_task(startRecording(stop_flag))
            #asyncio.create_task(pingWebSocket(stop_flag1, websocket))
        elif action_value =="stopRecording":
            stop_flag.set()
            global endTime
            endTime = getCurrentTime()

            converted_text=convertAudioToText()
            delete_audio_files()
            print("Converted Text:", converted_text)
            response = {"message": "stopped"}
            await websocket.send(json.dumps(response))
            summarized_content =summarizer(converted_text)
            print("Summarized content:", summarized_content)
            response = {"message": "summarized"}
            await websocket.send(json.dumps(response))
        else:
            sendMail(json_data['email'], summarized_content)
            response = {"message": "mailSent"}
            await websocket.send(json.dumps(response))
        
stop_flag = asyncio.Event()
#stop_flag1 = asyncio.Event()

start_server = websockets.serve(lambda ws, path: on_message(ws, path, stop_flag), "localhost", 8774)

await start_server

Received message from extension: {"action":"startRecording"}
startRecording
The current date and time is: 02:48:53
False
Recording loop 1 - 10 seconds...
Finished recording loop 1.
Audio saved to meeting_1.wav
False
Recording loop 2 - 10 seconds...
Finished recording loop 2.
Audio saved to meeting_2.wav
False
Recording loop 3 - 10 seconds...
Finished recording loop 3.
Audio saved to meeting_3.wav
Received message from extension: {"action":"stopRecording"}
stopRecording
The current date and time is: 02:49:27
Converting started
Converting file :  meeting_1.wav
Converting file :  meeting_2.wav
Converting file :  meeting_3.wav
Converted Text: 
hi this escape dinner so i am going to talk about my special talent so i love
singing and participating in many competitions in solid tamilnadu and a sung many songs
and the impersonating lyrics and i blake i got a place for those thank you
Generated Text: ['i am going to talk about my special talent so i am going to talk about my special talent so i